In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import os.path
import scipy.spatial.distance as sd
from skip_thoughts import configuration
from skip_thoughts import encoder_manager
from datetime import datetime, timedelta
from time import sleep
import pandas as pd
import re
from os import listdir
import random



VOCAB_FILE = "/home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/vocab.txt"
EMBEDDING_MATRIX_FILE = "/home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/embeddings.npy"
CHECKPOINT_PATH = "/home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/model.ckpt-501424"


# Set up the encoder. Here we are using a single unidirectional model.
# To use a bidirectional model as well, call load_model() again with
# configuration.model_config(bidirectional_encoder=True) and paths to the
# bidirectional model's files. The encoder will use the concatenation of
# all loaded models.
encoder = encoder_manager.EncoderManager()
encoder.load_model(configuration.model_config(),
                   vocabulary_file=VOCAB_FILE,
                   embedding_matrix_file=EMBEDDING_MATRIX_FILE,
                   checkpoint_path=CHECKPOINT_PATH)


/home/simone/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Reading vocabulary from /home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/vocab.txt
INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from /home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:Loading model from checkpoint: /home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/model.ckpt-501424
INFO:tensorflow:Restoring parameters from /home/simone/skip_thoughts/pretrained/skip_thoughts_uni_2017_02_02/model.ckpt-501424
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-501424


In [2]:
tickers = ['AAPL','AMZN','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX']     
tickers = ['AAPL']
for ticker in tickers:
    news = pd.read_csv('/home/simone/Desktop/NLFF/intrinioDataset/preprocessing/preprocessed/' + ticker + '.csv')
    news = news.sort_values(by=['PUBLICATION_DATE'])
    news = news.reset_index(drop=True)

    news['EMBEDDING'] = ''
    failed = 0

    for index, row in news.iterrows():
        if(not pd.isnull(news['SUMMARY'][index])):
            enc = encoder.encode([news['SUMMARY'][index]])
            if(not ( np.isnan(enc[0][0]) or pd.isnull(news['SUMMARY'][index] ))):     
                news.at[index, 'EMBEDDING'] = enc[0]
            else:
                failed += 1
                print('--------- Fail')
            if(index % 1000 == 0):
                print(index)

    print('Failed: '+str(failed))           

    news = news[['PUBLICATION_DATE', 'SUMMARY', 'EMBEDDING']]
    news.to_json(str(ticker)+'_EMBEDDING.json')

0


/home/simone/Desktop/NLFF/model/skip_thoughts/skip_thoughts_encoder.py:257: RuntimeWarning: invalid value encountered in true_divide
  thought_vectors = [v / np.linalg.norm(v) for v in thought_vectors]


--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
1000
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
2000
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
3000
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- Fail
--------- 

In [ ]:
a = list()
for v in news['EMBEDDING'].tolist():
    if not sum(v):
        a.append(v)
    else:
        a.append(v[0])


In [ ]:
#news = news[np.isfinite(a)]

news = news[[not pd.isnull(v) for v in news['EMBEDDING'].tolist()]]

In [ ]:
news[cols_to_keep].to_csv('AAPL_EMBEDDING.csv')

In [ ]:
# import pandas as pd
# news = pd.read_json('AAPL_EMBEDDING.json')

In [ ]:
# news['EMBEDDING'][0]

In [ ]:
for i, row in news.iterrows():
    print(news['EMBEDDING'][i][0])